# Week 04: Multi-class Classification

## Introduction


In this exercise, we will implement logistic regression based multiclass classification to recognize handwritten digits. 

In [ ]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# Module to load MATLAB mat datafile format (Input and output module of scipy)


# Python Imaging Library (PIL)


# tells matplotlib to embed plots within the notebook
%matplotlib inline

## Multi-class Classification

For this exercise, logistic regression will be used to recognize handwritten digits (from 0 to 9).

### Dataset

The data set is given in `Week4_data.mat` that contains 5000 training examples of handwritten digits. Use the function `loadmat` within the `scipy.io` module to load the data.

There are 5000 training examples in `Week4_data.mat`, where each training example is a 20 pixel by 20 pixel grayscale image of the digit. Each pixel is represented by a floating point number indicating the grayscale intensity at that location. The 20 by 20 grid of pixels is “unrolled” into a 400-dimensional vector. Each of these training examples becomes a single row in our data matrix `X`. This gives us a 5000 by 400 matrix `X` where every row is a training example for a handwritten digit image.

$$ X = \begin{bmatrix} - \: (x^{(1)})^T \: - \\ -\: (x^{(2)})^T \:- \\ \vdots \\ - \: (x^{(m)})^T \:-  \end{bmatrix} $$

The second part of the training set is a 5000-dimensional vector `y` that contains labels for the training set.

![](Figures/MultiClass.png)

![](Figures/reshape.jpg)

In [ ]:
# 10 labels, from 1 to 10 (note that we have mapped "0" to label 10)
num_labels = 10

data = 
X, y = 

# set the zero digit to 0, rather than its mapped 10 in this dataset


m = y.size

### MATLAB Data

![](data11.png)

### Definition of useful functions that are going to be used thoughout the code

In [ ]:
def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Displays 2D data stored in X in a nice grid.
    """
    # Compute rows, cols
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promote to a 2 dimensional array
    else:
        raise IndexError('Input X should be 1 or 2 dimensional.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    # Compute number of items to display
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

#=========================================================================================        
        
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

### Visualize the data

To visualize the data that you imported, randomly selects 100 rows from `X` and passes those rows to the `displayData` function.

In [ ]:
# Randomly select 100 data points to display
rand_indices = np.random.choice(m, 6, replace=False)
#rand_indices = np.arange(990,1010)
sel = X[rand_indices, :]
print(y[rand_indices])

displayData(sel)

### Loading an image and converting it into 1D array of size (1 X 400)

To load an arbitrary image, resizing it to 20 X 20 pixel image, converting it into a grayscale image and lastly storing as a an array

In [ ]:
an_image1 = Image.open("sample.jpg") # Loading image
an_image = an_image1.resize((20,20)) # Resizing

grayscale_image = an_image.convert("L") # Conversion into grayscale
grayscale_array = np.asarray(grayscale_image) # Conversion into an array

grayscale_1D = np.ravel(grayscale_array) # Conversion into 1D array

#pyplot.imshow(grayscale_array, cmap="gray")
displayData(grayscale_1D,figsize=(3,3))
print(grayscale_1D.shape)

<a id="section1"></a>
#### Vectorizing the cost function 

We will begin by writing a vectorized version of the cost function. Recall that in (unregularized) logistic regression, the cost function is

$$ J(w) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log \left( h_w\left( x^{(i)} \right) \right) - \left(1 - y^{(i)} \right) \log \left(1 - h_w \left( x^{(i)} \right) \right) \right] $$

To compute each element in the summation, we have to compute $h_w(x^{(i)})$ for every example $i$, where $h_w(x^{(i)}) = g(w^T x^{(i)})$ and $g(z) = \frac{1}{1+e^{-z}}$ is the sigmoid function. It turns out that we can compute this quickly for all our examples by using matrix multiplication. Let us define $X$ and $w$ as

$$ X = \begin{bmatrix} - \left( x^{(1)} \right)^T - \\ - \left( x^{(2)} \right)^T - \\ \vdots \\ - \left( x^{(m)} \right)^T - \end{bmatrix} \qquad \text{and} \qquad w = \begin{bmatrix} w_0 \\ w_1 \\ \vdots \\ w_n \end{bmatrix} $$

Then, by computing the matrix product $Xw$, we have: 

$$ Xw = \begin{bmatrix} - \left( x^{(1)} \right)^Tw - \\ - \left( x^{(2)} \right)^Tw - \\ \vdots \\ - \left( x^{(m)} \right)^Tw - \end{bmatrix} = \begin{bmatrix} - w^T x^{(1)}  - \\ - w^T x^{(2)} - \\ \vdots \\ - w^T x^{(m)}  - \end{bmatrix} $$

In the last equality, we used the fact that $a^Tb = b^Ta$ if $a$ and $b$ are vectors. This allows us to compute the products $w^T x^{(i)}$ for all our examples $i$ in one line of code.

#### Vectorizing the gradient

Recall that the gradient of the (unregularized) logistic regression cost is a vector where the $j^{th}$ element is defined as

$$ \frac{\partial J }{\partial w_j} = \frac{1}{m} \sum_{i=1}^m \left( \left( h_w\left(x^{(i)}\right) - y^{(i)} \right)x_j^{(i)} \right) $$

To vectorize this operation over the dataset, we start by writing out all the partial derivatives explicitly for all $w_j$,

$$
\begin{align*}
\begin{bmatrix} 
\frac{\partial J}{\partial w_0} \\
\frac{\partial J}{\partial w_1} \\
\frac{\partial J}{\partial w_2} \\
\vdots \\
\frac{\partial J}{\partial w_n}
\end{bmatrix} = &
\frac{1}{m} \begin{bmatrix}
\sum_{i=1}^m \left( \left(h_w\left(x^{(i)}\right) - y^{(i)} \right)x_0^{(i)}\right) \\
\sum_{i=1}^m \left( \left(h_w\left(x^{(i)}\right) - y^{(i)} \right)x_1^{(i)}\right) \\
\sum_{i=1}^m \left( \left(h_w\left(x^{(i)}\right) - y^{(i)} \right)x_2^{(i)}\right) \\
\vdots \\
\sum_{i=1}^m \left( \left(h_w\left(x^{(i)}\right) - y^{(i)} \right)x_n^{(i)}\right) \\
\end{bmatrix} \\
= & \frac{1}{m} \sum_{i=1}^m \left( \left(h_w\left(x^{(i)}\right) - y^{(i)} \right)x^{(i)}\right) \\
= & \frac{1}{m} X^T \left( h_w(x) - y\right)
\end{align*}
$$

where

$$  h_w(x) - y = 
\begin{bmatrix}
h_w\left(x^{(1)}\right) - y^{(1)} \\
h_w\left(x^{(2)}\right) - y^{(2)} \\
\vdots \\
h_w\left(x^{(m)}\right) - y^{(m)} 
\end{bmatrix} $$

Note that $x^{(i)}$ is a vector, while $h_w\left(x^{(i)}\right) - y^{(i)}$  is a scalar (single number).
To understand the last step of the derivation, let $\beta_i = (h_w\left(x^{(m)}\right) - y^{(m)})$ and
observe that:

$$ \sum_i \beta_ix^{(i)} = \begin{bmatrix} 
| & | & & | \\
x^{(1)} & x^{(2)} & \cdots & x^{(m)} \\
| & | & & | 
\end{bmatrix}
\begin{bmatrix}
\beta_1 \\
\beta_2 \\
\vdots \\
\beta_m
\end{bmatrix} = x^T \beta
$$

where the values $\beta_i = \left( h_w(x^{(i)} - y^{(i)} \right)$.

Now the job is to define a new function (`lrCostFunction`) which will take the data (vectors `X` and `y`) and parameter (`Lambda`) as input and return the cost as a scalar. 

#### Regularized logistic regression

Now add regularization to the cost function. Recall that for regularized logistic
regression, the cost function is defined as

$$ J(w) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log \left(h_w\left(x^{(i)} \right)\right) - \left( 1 - y^{(i)} \right) \log\left(1 - h_w \left(x^{(i)} \right) \right) \right] + \frac{\lambda}{2m} \sum_{j=1}^n w_j^2 $$

Note that $w_0$ should not be regularized as it is used as bias term.
Correspondingly, the partial derivative of regularized logistic regression cost for $w_j$ is defined as

$$
\begin{align*}
& \frac{\partial J(w)}{\partial w_0} = \frac{1}{m} \sum_{i=1}^m \left( h_w\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)}  & \text{for } j = 0 \\
& \frac{\partial J(w)}{\partial w_j} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_w\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m} w_j & \text{for } j  \ge 1
\end{align*}
$$

Once you finished your implementation, you can call the function `lrCostFunction` to test your solution using the following cell:

In [ ]:
def lrCostFunction(w, X, y, lambda_):
    
    
    
    
    
    
    
    return J, grad

<a id="section2"></a>
### Multi-class Classification

In this part of the exercise, we will implement multi-class classification by training multiple regularized logistic regression classifiers, one for each of the $K$ classes in our dataset.

We will code for the function `oneVsAll` below, to train one classifier for each class. In particular, the code should return all the classifier parameters in a matrix $w \in \mathbb{R}^{K \times (N +1)}$, where each row of $w$ corresponds to the learned logistic regression parameters for one class. One can do this with a “for”-loop from $0$ to $K-1$, training each classifier independently.

The obvious approach is to use a one-versus-the-rest approach (also called one-vs-all), in which we train C binary classifiers, fc(x), where the data from class c is treated as positive, and the data from all the other classes is treated as negative.

<a id="oneVsAll"></a>

In [ ]:
c = 0
print(y[490:510,])
c1 = (y == c)
print(c1[490:510,])

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
        
    m, n = X.shape
    
    all_w = np.zeros((num_labels, n + 1))

    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    
    
    
    
    return all_w

After complting the code for `oneVsAll`, the following cell shall use the code to train a multi-class classifier. 

In [ ]:
lambda_ = 0.1
all_w = oneVsAll(X, y, num_labels, lambda_)
print(all_w.shape)

<a id="section3"></a>
#### Multi-class Prediction

After training one-vs-all classifier, one can now use it to predict the digit contained in a given image. For each input, one should compute the “probability” that it belongs to each class using the trained logistic regression classifiers. The one-vs-all prediction function will pick the class for which the corresponding logistic regression classifier outputs the highest probability and return the class label (0, 1, ..., K-1) as the prediction for the input example.
<a id="predictOneVsAll"></a>

In [ ]:
def predictOneVsAll(all_w, X):
        
    m = X.shape[0];
    num_labels = all_w.shape[0]

    # Return the following variable 
    p = np.zeros(m)

    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    p = 
    
    return p

### Decoding the above function

In the example below, the size of variable `ff` will be 5000 $\times$ 10  $\textit{i.e.}$  using sigmoid function, the expression will compute the probability for all the classes (10 in total) for a particular example (5000 in total) by using the optimized parameters (`w_all`) . Then using the `argmax` function, the index corresponding to the maximum probability (among 10 probabilities for a particular example) will be picked up. The index will then represent the classification of the given example in one of the 10 classes.

Now, call `predictOneVsAll` function using the learned value of $w$. One should see the training set accuracy in percentage which shows that the algorithm classifies `p%` of the examples in the training set correctly.

In [ ]:
pred = predictOneVsAll(all_w, X)
print('Training Set Accuracy: {:.2f}%'.format(np.mean(pred == y) * 100))

In [ ]:
#rand_indices = np.arange(3331,3346)
rand_indices = np.random.choice(m, 20, replace=False)
#print(rand_indices)
print(pred[rand_indices])
sel = X[rand_indices, :]

displayData(sel)